In [1]:
# Import libraries
import pandas as pd
import numpy as np
import pandasql as psql

In [2]:
# Read teams
filename = '/Users/darionguan/Desktop/NBA Analytics/teams.csv'
teams_df = pd.read_csv(filename)

In [3]:
teams_df = teams_df[['TEAM_ID', 'ABBREVIATION']]
teams_df.head()

,TEAM_ID,ABBREVIATION
0,1610612737,ATL
1,1610612738,BOS
2,1610612740,NOP
3,1610612741,CHI
4,1610612742,DAL


In [4]:
nba_dict = teams_df.copy()
nba_dict.set_index('TEAM_ID',inplace=True)
nba_dict = nba_dict.to_dict()['ABBREVIATION']
nba_dict

{1610612737: 'ATL',
 1610612738: 'BOS',
 1610612740: 'NOP',
 1610612741: 'CHI',
 1610612742: 'DAL',
 1610612743: 'DEN',
 1610612745: 'HOU',
 1610612746: 'LAC',
 1610612747: 'LAL',
 1610612748: 'MIA',
 1610612749: 'MIL',
 1610612750: 'MIN',
 1610612751: 'BKN',
 1610612752: 'NYK',
 1610612753: 'ORL',
 1610612754: 'IND',
 1610612755: 'PHI',
 1610612756: 'PHX',
 1610612757: 'POR',
 1610612758: 'SAC',
 1610612759: 'SAS',
 1610612760: 'OKC',
 1610612761: 'TOR',
 1610612762: 'UTA',
 1610612763: 'MEM',
 1610612764: 'WAS',
 1610612765: 'DET',
 1610612766: 'CHA',
 1610612739: 'CLE',
 1610612744: 'GSW'}

In [5]:
# Query to get TEAM_ID
# q = """
# SELECT t.team_id AS TEAM_ID,
#         t.
# """
teams_df = teams_df[['TEAM_ID', 'ABBREVIATION']]
pac_teams = teams_df.loc[(teams_df['ABBREVIATION'] == 'GSW') |
                        (teams_df['ABBREVIATION'] == 'LAC') |
                        (teams_df['ABBREVIATION'] == 'LAL') |
                        (teams_df['ABBREVIATION'] == 'PHX') |
                        (teams_df['ABBREVIATION'] == 'SAC')]

pac_teams

,TEAM_ID,ABBREVIATION
7,1610612746,LAC
8,1610612747,LAL
17,1610612756,PHX
19,1610612758,SAC
29,1610612744,GSW


In [6]:
filename = '/Users/darionguan/Desktop/NBA Analytics/games.csv'
games_df = pd.read_csv(filename)

# Plus one for each season
# games_df['SEASON'] = games_df['SEASON'] + 1

# Convert to datetime format
games_df['GAME_DATE_EST'] = pd.to_datetime(games_df['GAME_DATE_EST'], format='%Y-%m-%d')

# Subset to 2015-2016 regular season
games_df = games_df.loc[(games_df['GAME_DATE_EST'] >= '2015-10-27') & (games_df['GAME_DATE_EST'] <= '2016-04-13')]

# Convert floats to int
games_df['PTS_home'] = games_df['PTS_home'].astype(int)
games_df['PTS_away'] = games_df['PTS_away'].astype(int)

games_df = games_df[['GAME_DATE_EST', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'PTS_home', 'PTS_away', 'HOME_TEAM_WINS']]

games_df = games_df.replace({'HOME_TEAM_ID': nba_dict})
games_df = games_df.replace({'VISITOR_TEAM_ID': nba_dict})

games_df

,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,PTS_home,PTS_away,HOME_TEAM_WINS
23048,2016-04-13,BOS,MIA,98,88,1
23049,2016-04-13,BKN,TOR,96,103,0
23050,2016-04-13,CHA,ORL,117,103,1
23051,2016-04-13,CLE,DET,110,112,0
23052,2016-04-13,WAS,ATL,109,98,1
...,...,...,...,...,...,...
24273,2015-10-28,SAC,LAC,104,111,0
24274,2015-10-28,LAL,MIN,111,112,0
24275,2015-10-27,ATL,DET,94,106,0
24276,2015-10-27,CHI,CLE,97,95,1


In [7]:
# Subset GSW home games
gsw_home = games_df.copy()
gsw_home = gsw_home.loc[(gsw_home['HOME_TEAM_ID'] == 'GSW')]
gsw_home.rename(columns={'HOME_TEAM_ID': 'GSW',
                         'VISITOR_TEAM_ID': 'OPP',
                         'PTS_home': 'GSW_PTS',
                         'PTS_away': 'OPP_PTS'}, inplace=True)
gsw_home.shape

(41, 6)

In [8]:
# Subset GSW away games
gsw_away = games_df.copy()
gsw_away = gsw_away.loc[(gsw_away['VISITOR_TEAM_ID'] == 'GSW')]
gsw_away.rename(columns={'HOME_TEAM_ID': 'OPP',
                         'VISITOR_TEAM_ID': 'GSW',
                         'PTS_home': 'OPP_PTS',
                         'PTS_away': 'GSW_PTS'}, inplace=True)
gsw_away.shape

(41, 6)

In [9]:
# Concatenate GSW home and away games
gsw_df = pd.concat([gsw_home, gsw_away])

# Subset
gsw_df = gsw_df[['GAME_DATE_EST', 'GSW', 'OPP', 'GSW_PTS', 'OPP_PTS']]

# Sort by date
gsw_df = gsw_df.sort_values(by='GAME_DATE_EST', ascending=True)

# Reset index
gsw_df = gsw_df.reset_index(drop=True)

# Create point differential
gsw_df['PTS_DIFF'] = gsw_df['GSW_PTS'] - gsw_df['OPP_PTS']

# Create GSW_WIN column
gsw_df['GSW_WIN'] = gsw_df['PTS_DIFF'] >= 1
gsw_df['GSW_WIN'] = gsw_df['GSW_WIN'].astype(int)

# Create GSW_WIN column
gsw_df['GSW_LOSE'] = gsw_df['PTS_DIFF'] < 0
gsw_df['GSW_LOSE'] = gsw_df['GSW_LOSE'].astype(int)

# Cumulate wins and losses
gsw_df['TOTAL_WINS'] = gsw_df['GSW_WIN'].cumsum()
gsw_df['TOTAL_LOSSES'] = gsw_df['GSW_LOSE'].cumsum()

# Create RECORD column
gsw_df['TOTAL_WINS'] = gsw_df['TOTAL_WINS'].astype(str)
gsw_df['TOTAL_LOSSES'] = gsw_df['TOTAL_LOSSES'].astype(str)
gsw_df['RECORD'] = gsw_df['TOTAL_WINS'] + '-' + gsw_df['TOTAL_LOSSES']
gsw_df['TOTAL_WINS'] = gsw_df['TOTAL_WINS'].astype(int)
gsw_df['TOTAL_LOSSES'] = gsw_df['TOTAL_LOSSES'].astype(int)

# Create GAME_NUM
gsw_df['GAMES'] = 1
gsw_df['GAME_NUM'] = gsw_df['GAMES'].cumsum()

# Compute WIN_STREAK
m = gsw_df.GSW_WIN.astype(bool)
gsw_df['WIN_STREAK'] = (m.groupby([m, (~m).cumsum().where(m)]).cumcount().add(1).mul(m))

gsw_df = gsw_df[['GAME_DATE_EST', 'OPP', 'GSW_PTS', 'OPP_PTS', 'PTS_DIFF', 'RECORD', 'GAME_NUM', 'WIN_STREAK']]

gsw_df.to_csv('gsw_games.csv', index=False)